<a href="https://colab.research.google.com/github/Juhwan01/DeepDive/blob/main/KoBERT%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_KorNLl%ED%92%80%EA%B8%B0(%EB%8B%A4%EC%A4%91_%ED%81%B4%EB%9E%98%EC%8A%A4_%EB%B6%84%EB%A5%98)_%EC%A7%81%EC%A0%91%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 이전과 똑같이 레거시 모드로
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import urllib.request
from sklearn import preprocessing
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
# 훈련 데이터 다운로드
# MultiNLI 한국어 훈련 데이터
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/multinli.train.ko.tsv",
                          filename="multinli.train.ko.tsv")

# SNLI 1.0 한국어 훈련 데이터
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/snli_1.0_train.ko.tsv",
                          filename="snli_1.0_train.ko.tsv")

# 검증 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.dev.ko.tsv",
                          filename="xnli.dev.ko.tsv")

# 테스트 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.test.ko.tsv",
                          filename="xnli.test.ko.tsv")

In [ ]:
# tsv -> 탭(\t)으로 구분된다 <=> csv는 기본적으로 쉼표(,))로 구분된다
# quoting=3 (QUOTE_NONE)으로 읽을 때
# 따옴표가 제거되지 않고 그대로 유지됨
# "Hello"도, "Quoted text"도 따옴표 포함 그대로 읽음 -> 따옴표를 그대로 보존해야 할 때 사용
train_snli = pd.read_csv("snli_1.0_train.ko.tsv", sep='\t', quoting=3)
train_xnli = pd.read_csv("multinli.train.ko.tsv", sep='\t', quoting=3)
val_data = pd.read_csv("xnli.dev.ko.tsv", sep='\t', quoting=3)
test_data = pd.read_csv("xnli.test.ko.tsv", sep='\t', quoting=3)

In [ ]:
# 위에서 읽어온 데이터는 DataFrame형태 이 두개의 객체를 세로(행 방향)로 붙인다.
# pandas 2.0부터 DataFrame.append() 메서드가 삭제 되었습니다. -> concat 메서드 사용
#ignore_index=True는 기존 인덱스를 무시하고 새로운 연속적인 인덱스(0, 1, 2, ...)로 다시 만들어주는 옵션입니다.
train_data = pd.concat([train_snli,train_xnli])
# 결합 후 섞기
# sample() 함수는 DataFrame에서 무작위 샘플을 추출하는 함수
# frac=1 은 전체 데이터(frac=1 즉 100%)를 무작위로 섞어 반환한다는 뜻
# frac	데이터에서 뽑을 비율(0~1 사이 실수)	frac=0.5 → 50% 무작위 추출
# 원본 인덱스는 유지되기 때문에, 만약 인덱스도 재정렬하고 싶으면 reset_index(drop=True)를 추가로 써야 합니다.
train_data = train_data.sample(frac=1)

In [ ]:
train_data.head()

In [ ]:
val_data.head()

In [ ]:
test_data.head()

In [ ]:
def drop_na_and_duplciates(df):
  df = df.dropna() # how='any'가 기본값
  df = df.drop_duplicates()
  # drop=True를 쓰면 기존 인덱스를 새로운 컬럼으로 추가하지 않고 그냥 버립니다. 만약 drop=False면, 기존 인덱스가 컬럼으로 남는다
  df = df.reset_index(drop=True) # DataFrame의 인덱스를 초기화
  return df

In [ ]:
# 데이터 전처리
train_data = drop_na_and_duplciates(train_data)
val_data = drop_na_and_duplciates(val_data)
test_data = drop_na_and_duplciates(test_data)

In [ ]:
print('훈련용 샘플 개수 :', len(train_data))
print('검증용 샘플 개수 :', len(val_data))
print('테스트용 샘플 개수 :', len(test_data))

In [ ]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

In [ ]:
max_seq_len = 128
# iloc -> 데이터프레임에서 위치 기반으로 행을 가져온다 -> iloc[0]은 첫 번째 행
# 왜 [0] 사용안하느냐?
# 인덱스가 재설정되지 않았거나 섞인 경우에는 iloc[0]이 더 안전하고 정확
sent1 = train_data['sentence1'].iloc[0]
sent2 = train_data['sentence2'].iloc[0]

print('문장1 :',sent1)
print('문장2 :',sent2)

In [ ]:
# encode_plus 메서드는 문장 쌍 관계 처리를 위해 만들어졌다.
# 이렇게 한번에 인코딩하여 두 문장을 이어서 모델에 넣는다.
# padding='max_length' -> 0으로 max_length까지 패딩
# BertTokenizerFast랑은 다르게 작동하는듯
encoding_result = tokenizer.encode_plus(sent1,sent2,max_length=max_seq_len,padding='max_length',truncation=True)

In [ ]:
print(encoding_result['attention_mask'])

In [ ]:
# 위에서한 전처리를 모든 데이터에 적용
def convert_examples_to_features(sent_list1,sent_list2,max_seq_len,tokenizer):
  input_ids, attention_masks, token_type_ids = [], [], []
  # tqdm -> 진행도
  # zip(sent1,sent2) -> zip()으로 묶으면 두 리스트를 쌍으로 묶어서 튜플을 만든다
  # 예시 -> [('안녕', '하세요'), ('좋은', '아침'), ('나쁜', '날씨')]
  for sent1,sent2 in tqdm(zip(sent_list1,sent_list2),total=len(sent_list1)):
    encoding_result = tokenizer.encode_plus(sent1,sent2,max_length=max_seq_len,padding='max_length',truncation=True)
    # 모델에 넣을 입력들
    input_ids.append(encoding_result['input_ids'])
    attention_masks.append(encoding_result['attention_mask'])
    token_type_ids.append(encoding_result['token_type_ids'])

  # 최종 모델 입력을 위해 리스트를 numpy 배열(int 타입)로 변환
  # 모델 입력은 일반적으로 numpy 배열이나 Tensor 형태여야 한다.(이유->이전 실습 노트에 작성)
  input_ids = np.array(input_ids, dtype=int)
  attention_masks = np.array(attention_masks, dtype=int)
  token_type_ids = np.array(token_type_ids, dtype=int)

  return (input_ids,attention_masks,token_type_ids)

In [ ]:
X_train = convert_examples_to_features(train_data['sentence1'],train_data['sentence2'],max_seq_len,tokenizer)

In [ ]:
input_id = X_train[0][0]
attention_mask = X_train[1][0]
token_type_id = X_train[2][0]

print('단어에 대한 정수 인코딩 :', input_id)
print('어텐션 마스크 :', attention_mask)
print('세그먼트 인코딩 :', token_type_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :', tokenizer.decode(input_id))

In [ ]:
# 검증 데이터도 똑같이 진행
X_val = convert_examples_to_features(val_data['sentence1'],val_data['sentence2'],max_seq_len,tokenizer)

In [ ]:
input_id = X_val[0][0]
attention_mask = X_val[1][0]
token_type_id = X_val[2][0]

print('단어에 대한 정수 인코딩 :', input_id)
print('어텐션 마스크 :', attention_mask)
print('세그먼트 인코딩 :', token_type_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :', tokenizer.decode(input_id))

In [ ]:
# 테스트 데이터도 똑같이 진행
X_test = convert_examples_to_features(test_data['sentence1'],test_data['sentence2'],max_seq_len,tokenizer)

In [ ]:
input_id = X_test[0][0]
attention_mask = X_test[1][0]
token_type_id = X_test[2][0]

print('단어에 대한 정수 인코딩 :', input_id)
print('어텐션 마스크 :', attention_mask)
print('세그먼트 인코딩 :', token_type_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :', tokenizer.decode(input_id))

In [ ]:
# 문자열로 구성된 레이블 정수 인코딩 진행
train_label = train_data['gold_label'].tolist()
val_label = val_data['gold_label'].tolist()
test_label = test_data['gold_label'].tolist()

# LabelEncoder는 문자열 라벨을 숫자로 변환해주는 도구
#fit(): 카테고리 종류 학습
#transform(): 숫자로 변환
#fit_transform(): fit + transform 한번에
#inverse_transform(): 숫자 -> 원본으로 역변환
idx_encode = preprocessing.LabelEncoder()
idx_encode.fit(train_label)

# 정수 변환
y_train = idx_encode.transform(train_label)
y_val = idx_encode.transform(val_label)
y_test = idx_encode.transform(test_label)
# idx_encode.classes_
# array(['contradiction', 'entailment', 'neutral'], dtype=object)
# label_idx = dict(zip(...))
# dict(zip(['contradiction', 'entailment', 'neutral'], [0, 1, 2]))
# 결과 적으로 -> 'contradiction': 0
label_idx = dict(zip((list(idx_encode.classes_)),idx_encode.transform(list(idx_encode.classes_))))
idx_label = {value:key for key,value in label_idx.items()}
print('각 레이블과 정수 :',label_idx)

In [ ]:
print('변환 전 :', train_label[:5])
print('변환 후 :', y_train[:5])

In [ ]:
# 이전에 정의해둔 클래스 복붙(앞에서 한 내용 bert 네이버 영화 리뷰 분류) -> 차이점은(다중분류)
#
class TFBertForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super().__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        # 출력층을 하나 만들어서 classifier에 저장
        # 출력 뉴런 하나로 이진 분류 예측 -> 출력 뉴런 num_labels수 만큼 = 다중 분류 문제
        self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                # softmax사용 -> 다중 분류
                                                activation='softmax',
                                                name='classifier')
    def call(self, inputs):
      input_ids, attention_mask, token_type_ids = inputs

      # BERT 모델을 통과시켜 모든 토큰에 대한 출력 및 [CLS] 벡터 얻기
      outputs = self.bert(input_ids=input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids)

      # 문장의 대표 벡터인 [CLS] 토큰의 벡터 가져오기 (pooled_output)
      cls_token = outputs[1]

      # [CLS] 벡터를 Dense 레이어에 넣어서 다중 클래스 확률 예측
      # softmax를 사용하므로 각 클래스에 대한 확률 분포가 나옴
      prediction = self.classifier(cls_token)

      return prediction

In [ ]:
# 앞이랑 다르게 label이 3개이기 때문에 num_labels=3
model = TFBertForSequenceClassification("klue/bert-base",num_labels=3)

In [ ]:
optimizer = tf.keras.optimizers.Adam(5e-5)
# SparseCategoricalCrossentropy -> 레이블 형식: 정수 인코딩된 레이블 (예: 2, 0, 1, ...)
# 다중 클래스 분류에서 레이블이 정수 인코딩 상태이면 SparseCategoricalCrossentropy 를 쓰고,
# 레이블이 원-핫 인코딩 상태이면 CategoricalCrossentropy 를 씁니다.
loss = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.001,
    patience=2,
)
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=2,
    batch_size=32,
    callbacks=[early_stopping],
)

In [ ]:
# [0] 은 손실값
# [1] 은 첫 번째 평가 지표 (보통 accuracy)
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test, batch_size=1024)[1]))